In [1]:
import sys
!pip install bs4
!pip install requests
!pip install ta
!pip install finta
!pip install mplfinance
!pip install yfinance
!pip install ta-lib
!pip install numba
!pip install bokeh

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from numba import njit, jit 
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool
from bokeh.io import output_notebook # enables plot interface in J notebook


# Getting Data

## Importing from Yahoo Finance

In [34]:
oil_data = yf.download('CL=F')
oil_data = oil_data[1336:3100]
oil_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-01-03,61.040001,63.799999,60.810001,63.139999,63.139999,130635
2006-01-04,63.000000,63.650002,62.259998,63.419998,63.419998,105194
2006-01-05,63.400002,63.799999,62.599998,62.790001,62.790001,104035
2006-01-06,62.599998,64.449997,62.599998,64.209999,64.209999,110763
2006-01-09,64.150002,64.610001,62.900002,63.500000,63.500000,115558
...,...,...,...,...,...,...
2012-12-28,91.150002,91.489998,90.320000,90.800003,90.800003,131374
2012-12-31,90.410004,91.989998,90.000000,91.820000,91.820000,119525
2013-01-02,91.779999,93.870003,91.559998,93.120003,93.120003,203865


## Converting prices from dataframe to a numpy array

In [35]:
Open = oil_data['Open'].values.astype(np.float32)
High = oil_data['High'].values.astype(np.float32)
Low = oil_data['Low'].values.astype(np.float32)
Close = oil_data['Close'].values.astype(np.float32)
Volume = oil_data['Volume'].values.astype(np.float32)

# Indicators

## SMMA

In [45]:
@njit
def smma(arr, period):
    """
        Documentation:
        Smoothed moving average is a moving average that deals with a longer period, allowing for an
        easier price calculation and viewing and represents the combination of simple moving average
        and exponential moving average. A smoothed moving average does not refer to a fixed period,
        but rather collects and enrolls all available data from the past. To calculate today’s moving
        average, you have to subtract the yesterday’s smoothed moving average from today’s price. After
        that, you have to add the result to yesterday’s price.

        The SMMA formula

        The formula to calculate the SMMA is

        SMMA = (SMMA# – SMMA* + CLOSE)/N
        Where

        SMMA# – Previous bar’s smoothed sum

        SMMA* – Previous bar’s smoothed moving average
        CLOSE – Present closing price

        N – Period of smoothing

        :param period:
        :param arr: numpy array, dtype=np.float32, this is the close time series you want to use
        :param window: integer value, the bigger the value the slower the calculation will but also less impacted by
        outlayers, picking a good value according to the temporality is a most.

        :return: numpy array, dtype=np.float32, this is the value for every moment of the calculation.
     """

    alpha = 1 / period
    smma_calc = np.zeros(len(arr))
    smma_calc[period - 1] = np.nanmean(arr[0:period])
    for i in range(period, len(smma_calc)):
        smma_calc[i] = (arr[i] - smma_calc[i - 1]) * alpha + smma_calc[i - 1]
        smma_calc[:period - 1] = np.nan
    smma_calc = smma_calc.astype(np.float32)
    return smma_calc


## RSI

In [51]:
@njit
def rsi(close, window):
    """
    Documentation:
    The Relative Strength Index is a momentum oscillator that measures the speed and change of price movements.
    RSI = 100 - (100 / (1 + RS))
    RS = Average gain / Average loss

    The very first calculations for average gain and average loss are simple 14-period averages:

        - First Average Gain = Sum of Gains over the past 14 periods / 14.
        - First Average Loss = Sum of Losses over the past 14 periods / 14.

    The second, and subsequent, calculations are based on the prior averages and the current gain loss:

        - Average Gain = [(previous Average Gain) x 13 + current Gain] / 14.
        - Average Loss = [(previous Average Loss) x 13 + current Loss] / 14.


    Interpretation:
    RSI oscillates between zero and 100 and is considered overbought when above 70 and oversold when below 30.

    Example:
    Observe the last 14 closing prices of a stock.
    Determine whether the current day’s closing price is higher or lower than the previous day.
    Calculate the average gain and loss over the last 14 days.
    Compute the relative strength (RS): (AvgGain/AvgLoss)
    Compute the relative strength index (RSI): (100–100 / ( 1 + RS))

    The RSI will then be a value between 0 and 100.

    :param close: numpy array, dtype=np.float32, this is the close time series you want to use
    :param window: integer value, the bigger the value the slower the calculation will but also less impacted by
    outlayers, picking a good value according to the temporality is a most.

    :return: numpy array, dtype=np.float32, this is the value for the rsi.
    """
    shifted = np.roll(close, 1)[1:]
    shifted = np.append(np.zeros(1) + np.nan, shifted)

    delta = close - shifted
    gain, loss = np.copy(delta), np.copy(delta)
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    loss = np.absolute(loss)

    avg_gain = smma(gain.astype(np.float32), np.int32(window))
    avg_loss = smma(loss.astype(np.float32), np.int32(window))

    n = len(avg_loss)
    rsi_value = np.zeros(n) + np.nan
    for i in range(n):
        if avg_loss[i] == 0:
            rsi_value[i] = 100
        else:
            rs = avg_gain[i] / avg_loss[i]
            rsi_value[i] = 100 - (100 / (1 + rs))

    rsi_value[0: window] = np.nan

    return rsi_value.astype(np.float32)


## CCI

In [38]:
@njit
def cci(high, low, close, window):
    """
    TODO:

    Documentation:
    The Commodity Channel Index (CCI) is a versatile indicator that can be used to identify a new
    trend or warn of extreme conditions. Lambert originally developed CCI to identify cyclical turns in commodities,
    but the indicator can be successfully applied to indices, ETFs, stocks and other securities. In general,
    CCI measures the current price level relative to an average price level over a given period of time.

    Calculation:
    CCI = (Typical Price  -  N-period SMA of TP) / (.015 x Mean Deviation)

    Typical Price (TP) = (High + Low + Close)/3

    Constant = .015

    There are four steps to calculating the Mean Deviation:
    First, subtract the most recent N-period average of the typical price from each period's typical price.
    Second, take the absolute values of these numbers.
    Third, sum the absolute values.
    Fourth, divide by the total number of periods (N).

    Interpretation:
    CCI measures the difference between a security's price change and its average price change. High
    positive readings indicate that prices are well above their average, which is a show of strength. Low negative
    readings indicate that prices are well below their average, which is a show of weakness.

    The Commodity Channel Index (CCI) can be used as either a coincident or leading indicator. As a coincident
    indicator, surges above +100 reflect strong price action that can signal the start of an uptrend. Plunges below
    -100 reflect weak price action that can signal the start of a downtrend.

    :param high: numpy array, this is the high price of the candle
    :param low: numpy array, the low price of the candle
    :param close: numpy array, the closing price of the candle
    :param window: integer, the parameter for the moving averages in the calculation
    :return: np.array the values of the CCI
    """
    tp = np.zeros(len(high))
    for i in range(0, len(high)):
        tp[i] = (high[i] + low[i] + close[i]) / 3

    atp = np.zeros(len(high))  # average typical price
    md = np.zeros(len(high))  # mean deviation
    cci_value = np.zeros(len(high))
    for i in range(window - 1, len(high)):
        atp[i] = np.sum(tp[i - (window - 1):i + 1]) / window
        md[i] = np.sum(np.fabs(atp[i] - tp[i - (window - 1):i + 1])) / window
        if md[i] == 0:
            md[i] = np.NaN
        cci_value[i] = (tp[i] - atp[i]) / (0.015 * md[i])
    return cci_value.astype(np.float32)


In [61]:
## CUMMAX

In [62]:
@njit
def cummax(arr, window):
    """
               Documentation:
               Returns a vector whose elements are the cumulative max, products of the elements of
               the argument.
               :param arr: numpy array, dtype=np.float32, this is the close time series you want to use
               :param window: integer value.
               :return: numpy array, dtype=np.float32, this is the value for every moment of the calculation.
            """

    values = np.array([arr[i:i + window].max() for i in range(len(arr) - window + 1)])
    cummax_result = np.append(np.zeros(window - 1) + np.nan, values)
    return cummax_result.astype(np.float32)


## Strategy construction

### Parameters setup

In [82]:
window_observation = 10
rsi_overbought_level = 70
rsi_oversold_level = 30

In [74]:
oil_rsi = rsi(Close, 14)

In [75]:
oil_cci = cci(High, Low, Close, 20)
oil_cci

array([  0.      ,   0.      ,   0.      , ..., 175.04015 , 148.0374  ,
       119.235306], dtype=float32)

In [ ]:
rsi_overbought_condition = oil_rsi > rsi_overbought_level
    first_condition = cummax(rsi_overbought_level, window_observation)
    second_condition = oil_rsi < rsi_oversold_level
    third_condition = oil_cci > np.int32(100)
    exit_condition = oil_cci < 0

In [89]:
rsi_overbought_condition = oil_rsi > rsi_overbought_level

In [90]:
first_condition = cummax(rsi_overbought_condition, window_observation)

In [91]:
second_condition = oil_rsi < rsi_oversold_level

In [92]:
third_condition = oil_cci > np.int32(100)

In [94]:
exit_condition = oil_cci < 0

# Graphic Validation

In [81]:
# init bokeh
output_notebook()

# Candlestick
inc = Close > Open
dec = Close < Open

candle_size = 0.15

w = (17 * 30 * 30 * 25) * candle_size

is_data = pd.DataFrame(oil_data)

is_data['date'] = is_data.index

is_data.columns= is_data.columns.str.lower()

is_data = is_data[['date', 'open', 'high', 'low', 'close', 'volume']]

is_data['oil_rsi'] = oil_rsi
is_data['oil_cci'] = oil_cci

# print(is_data)

source_is_data = ColumnDataSource(is_data)

crosshair = CrosshairTool(dimensions='both')


p = figure(x_axis_type="datetime", plot_height=500, plot_width=1000,
           title=f'Symbol: CL=F, '
                 f'Timeframe: 1D ')


bar_high_low = p.segment("date", "high", "date", "low", color="black", source=source_is_data)
bar_high_low = p.segment("date", "high", "date", "low", color="black", source=source_is_data)
bar_open = p.vbar(is_data.date[inc], w, is_data.open[inc], is_data.close[inc], fill_color="green",
                  line_color="green")
bar_close = p.vbar(is_data.date[dec], w, is_data.open[dec], is_data.close[dec], fill_color="red",
                   line_color="red")

# Axis
p.xaxis.axis_label = 'TIME'
p.yaxis.axis_label = 'PRICE'

p_aux = figure(x_axis_type="datetime", plot_height=350, plot_width=1000, title= 'RSI', x_range=p.x_range)

p_aux_1 = figure(x_axis_type="datetime", plot_height=350, plot_width=1000, title= 'CCI', x_range=p.x_range)

line1 = p_aux.line("date", "oil_rsi", line_color="purple", legend_label='OIL RSI', line_width=1,
                   source=source_is_data)

line2 = p_aux_1.line("date", "oil_cci", line_color="black", legend_label='OIL CCI', line_width=1,
                     source=source_is_data)

line1_hover = HoverTool(
    renderers=[line1],
    tooltips=[
        ('date', '@date{%Y-%m-%d %H:%M:%S}'),
        ('oil_rsi', '@oil_rsi{0.00}'),
    ],

    formatters={
        '@date': 'datetime',  # use 'datetime' formatter for '@date' field
        # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

line2_hover = HoverTool(
    renderers=[line2],
    tooltips=[
        ('date', '@date{%Y-%m-%d %H:%M:%S}'),
        ('oil_cci', '@oil_cci{0.00}'),
    ],

    formatters={
        '@date': 'datetime',  # use 'datetime' formatter for '@date' field
        # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

crosshair = CrosshairTool(dimensions='both')

# Tools
p.add_tools(crosshair)
p_aux.add_tools(line1_hover, crosshair)
p_aux_1.add_tools(line2_hover, crosshair)


g = gridplot([[p], [p_aux], [p_aux_1]], sizing_mode='scale_width')
show(g)



Loading BokehJS ...